In [1]:
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver, AcadosSim, AcadosSimSolver
import casadi as ca
from initParam import * # constants and useful functions
import numpy as np
from scipy.linalg import block_diag
from tqdm import tqdm

# to make the figures interactive
import matplotlib

# to visualize videos in the notebook
from IPython.display import Video

In [2]:
def fiala(alpha, Fx, Fz, mu, Cy):
    # Ensure symbolic compatibility
    Fy_max = ca.sqrt(mu**2 * Fz**2 - Fx**2)
    alphas = ca.atan(3 * Fy_max / Cy)

    tan_alpha = ca.tan(alpha)
    abs_tan_alpha = ca.fabs(tan_alpha)
    sign_alpha = ca.sign(alpha)

    # Compute nonlinear region (|alpha| < alphas)
    Fy_linear = Cy * tan_alpha \
                - (Cy**2 * abs_tan_alpha * tan_alpha) / (3 * Fy_max) \
                + (Cy**3 * tan_alpha**3) / (27 * Fy_max**2)

    # Saturation region (|alpha| ≥ alphas)
    Fy_saturated = Fy_max * sign_alpha

    # Conditional expression using casadi.if_else
    Fy = ca.if_else(ca.fabs(alpha) < alphas, Fy_linear, Fy_saturated)

    return Fy

In [3]:
# tanh approximation
def fiala_tanh(alpha, Fx, Fz, mu, Cy):
    # Avoid invalid sqrt by clamping inside sqrt to >= 0
    Fy_max = ca.sqrt(ca.fmax(0, mu**2 * Fz**2 - Fx**2))
    alphas = ca.atan(Fy_max / Cy + 1e-6)  # avoid division by zero
    return Fy_max * ca.tanh(alpha / (alphas + 1e-6))  # avoid divide-by-zero

In [4]:
def STM_model()-> AcadosModel:
    
    #set model name
    model_name = "STM_model"
    nx = 3 # differential states: u, v, r
    nu = 2 # inputs: Fx, delta

    # Differential states 
    vx       = ca.SX.sym('vx', 1, 1)  # longitudinal velocity of the car 
    vy       = ca.SX.sym('vy', 1, 1)  # lateral velocity of the car
    r        = ca.SX.sym('r', 1, 1)  # yaw rate of the car

    x = ca.vertcat(vx,vy,r)  # state vector

    # Inputs
    Fx       = ca.SX.sym('Fx', 1, 1)  # longitudinal force of the car
    delta    = ca.SX.sym('delta', 1, 1)  #

    u = ca.vertcat(Fx, delta)  # input vector

    # setup symbolic variables for xdot (to be used with IRK integrator)
    vx_dot   = ca.SX.sym('vx_dot', 1, 1)
    vy_dot   = ca.SX.sym('vy_dot', 1, 1)
    r_dot    = ca.SX.sym('r_dot', 1, 1)

    xdot = ca.vertcat(vx_dot, vy_dot, r_dot)

    # define dynamics
    alpha_f = delta - ca.arctan2(vy + a*r, vx) # slip angle front
    alpha_r = -ca.arctan2(vy - b*r, vx) # slip angle rear
    Fyf = fiala_tanh(alpha_f, 0.0, Fz_Front, mu_f, Cyf) # lateral force front
    Fyr = fiala_tanh(alpha_r, Fx, Fz_Rear, mu_r, Cyr) # lateral force rear

    d_vx = (Fx - Fyf * ca.sin(delta)) / m + r * vy
    d_vy = (Fyf * ca.cos(delta) + Fyr) / m - r * vx
    d_r = (a * Fyf * ca.cos(delta) - b * Fyr) / J_CoG

    
    # explicit ODE right hand side (to be used with ERK integrator)
    f_expl = ca.vertcat(d_vx, 
                        d_vy, 
                        d_r)

    # implicit dynamics (to be used with IRK integrator)
    f_impl = xdot - f_expl

    # create acados model and fill in all the required fields
    model = AcadosModel()

    model.f_impl_expr = f_impl
    model.f_expl_expr = f_expl
    model.x = x
    model.xdot = xdot
    model.u = u
    model.name = model_name

    return model

In [5]:
def clear_previous_simulation():
    variables = globals().copy()
    for v in variables:
        if(type(variables[v]) is AcadosSimSolver or type(variables[v]) is AcadosOcpSolver):
            del globals()[v]

In [ ]:
def create_ocp_solver_description(model, N, T, x0) -> AcadosOcp:

    # create ocp object to formulate the OCP
    ocp = AcadosOcp()

    # define system dynamics model
    ocp.model = model

    # set prediction horizon:
    # tf - prediction horizon length [s]
    # N  - number of intervals in which the prediction horizon is divided 
    ocp.solver_options.tf = T
    ocp.dims.N = N
    ocp.solver_options.N_horizon = N
    
    # get state, control and cost dimensions
    nx = model.x.rows()
    nu = model.u.rows()

    ny = nx + nu 
    ny_e = nx
   
    # define cost type
    ocp.cost.cost_type = 'LINEAR_LS'
    ocp.cost.cost_type_e = 'LINEAR_LS'

    # define cost weigth matrices
    Q = np.diag([1, 10, 20])
    R = np.diag([1, 1])
    
    ocp.cost.W = block_diag(Q, R)
    ocp.cost.W_e = T/N*Q

    # define matrices characterizing the cost
    ocp.cost.Vx = np.vstack((np.eye(nx), np.zeros((nu, nx))))
    ocp.cost.Vu = np.vstack((np.zeros((nx, nu)), np.eye(nu)))
    ocp.cost.Vx_e = np.eye(nx)
    
    # alternatively, for the NONLINEAR_LS cost type
    #ocp.model.cost_y_expr = ca.vertcat(model.x, model.u)
    #ocp.model.cost_y_expr_e = model.x

    # initialize variables for reference
    ocp.cost.yref = np.zeros((ny,))
    ocp.cost.yref_e = np.zeros((ny_e,))

    # set constraints
    ocp.constraints.idxbu = np.array([0,1]) # indices of the control inputs to be constrained
    ocp.constraints.lbu = np.array([-80, -np.deg2rad(20)])  # lower bounds on inputs
    ocp.constraints.ubu = np.array([80, np.deg2rad(20)])   # upper bounds on inputs

    # ocp.constraints.idxbu = np.array([1]) # indices of the control inputs to be constrained
    # ocp.constraints.lbu = np.array([-np.deg2rad(20)])  # lower bounds on inputs
    # ocp.constraints.ubu = np.array([np.deg2rad(20)])   # upper bounds on inputs
  

    # initialize constraint on initial condition
    ocp.constraints.x0 = x0

    # set solver options
    # ocp.solver_options.print_level = 1
    ocp.solver_options.qp_solver = "PARTIAL_CONDENSING_HPIPM"  #FULL_CONDENSING_QPOASES, PARTIAL_CONDENSING_HPIPM
    ocp.solver_options.hessian_approx = "GAUSS_NEWTON"
    ocp.solver_options.integrator_type = "ERK"
    ocp.solver_options.nlp_solver_type = "SQP" #SQP, SQP_RTI

    # to configure partial condensing
    #ocp.solver_options.qp_solver_cond_N = int(N/10)

    # some more advanced settings (refer to the documentation to see them all)
    # - maximum number of SQP iterations (default: 100)
    ocp.solver_options.nlp_solver_max_iter = 100
    # - maximum number of iterations for the QP solver (default: 50)
    ocp.solver_options.qp_solver_iter_max = 50

    # - configure warm start of the QP solver (0: no, 1: warm start, 2: hot start)
    # (depends on the specific solver)
    ocp.solver_options.qp_solver_warm_start = 0
    
    return ocp


In [ ]:
clear_previous_simulation()
# Equilibrium point (found in matlab)

vx_eq = 3.65 # [m/s] longitudinal velocity
vy_eq = -1.70 # [m/s] lateral velocity
r_eq = 1.15 # [rad/s] yaw rate
delta_eq = 0.23 # [rad] steering angle 
Fx_eq = 53.57 # [N] longitudinal force

# define simulation fundamental time step [s]
ts_sim = 0.001

# model used to simulate the system
sim_model = STM_model()

# initial condition
x0 = np.array([vx_eq, vy_eq,r_eq])

# setup controller parameters
# - system model
model = STM_model()
# - controller sample time [s]
Ts = 0.01
# - number of shooting time intervals 
N  = 50
# - prediction horizon length [s]
T = N*Ts

T_tot = 10 # total simulation time [s]

# get state and control dimensions
nx, nu = model.x.rows(), model.u.rows()

# define reference 
# - define reference for the angle and, accordingly, the simulation time Tf
# angle_ref, Tf = piecewise_constant(np.array([np.pi, 0]), np.array([5, 10]), Ts)

vx_ref, Tf = piecewise_constant(np.array([vx_eq]), np.array([T_tot]), Ts)
vy_ref, _  = piecewise_constant(np.array([vy_eq]), np.array([T_tot]), Ts)
r_ref, _   = piecewise_constant(np.array([r_eq]), np.array([T_tot]), Ts)
Fx_ref, _   = piecewise_constant(np.array([Fx_eq]), np.array([T_tot]), Ts)
delta_ref, _   = piecewise_constant(np.array([delta_eq]), np.array([T_tot]), Ts)

# - provide a reference for all variables 
# y_ref = np.column_stack((np.zeros((len(angle_ref), 1)), angle_ref.reshape(-1,1), np.zeros((len(angle_ref), nx+nu-2))))
y_ref_nolookahead = np.column_stack((vx_ref, vy_ref, r_ref, Fx_ref, delta_ref))
# y_ref_nolookahead = np.column_stack((vx_ref, vy_ref, r_ref, np.zeros((len(vx_ref), nu))))

# - add N samples at the end (replicas of the last sample) for reference look-ahead
y_ref = np.vstack((y_ref_nolookahead, np.repeat(y_ref_nolookahead[-1].reshape(1,-1), N, axis=0)))

# compute the number of steps for simulation
N_steps, N_steps_dt, n_update = compute_num_steps(ts_sim, Ts, Tf)

# configure whether to apply shifting and to enable reference look-ahead
shifting    = False
ref_preview = False

In [8]:
# setup simulation of system dynamics
sim = AcadosSim()
sim.model = sim_model
sim.solver_options.T = ts_sim
sim.solver_options.integrator_type = 'ERK'

acados_integrator = AcadosSimSolver(sim, verbose=False)

# create OCP solver
ocp = create_ocp_solver_description(model, N, T, x0)
acados_ocp_solver = AcadosOcpSolver(ocp, verbose=False)

# initialize solver
for stage in range(N):
    acados_ocp_solver.set(stage, "x", x0)
    acados_ocp_solver.set(stage, "u", np.zeros((nu,)))

acados_ocp_solver.set(N, "x", x0)


# define iteration counter for the discrete-time part of the control loop
k = 0

# create variables to store state and control trajectories 
simX = np.zeros((N_steps + 1, nx))
simU = np.zeros((N_steps_dt, nu))
# set intial state
simX[0, :] = x0

# create variables to store, at each iteration, previous optimal solution
x_opt = np.zeros((N+1, nx, N_steps_dt + 1))
x_opt[:, :, 0] = np.repeat(x0.reshape(1,-1),N+1, axis=0)
u_opt = np.zeros((N, nu, N_steps_dt + 1))

# variable to store total CPU time
cpt = np.zeros((N_steps_dt,))


# simulation loop
for i in tqdm(range(N_steps), desc="Simulation", ascii=False, ncols=75, colour='green'):

    # check whether to update the discrete-time part of the loop
    if(i % n_update == 0):
        # update reference
        for j in range(N):
                acados_ocp_solver.set(j, "yref", y_ref[k + (j if ref_preview else 0), :])
        acados_ocp_solver.set(N, "yref", y_ref[k + (N if ref_preview else 0), 0:-nu])

        
        # if performing shifting, explicitly initialize solver
        # (otherwise, it will be automatically intialized with the previous solution)
        if shifting:
            for stage in range(N):
                acados_ocp_solver.set(stage, "x", x_opt[stage+1, :, k])
                acados_ocp_solver.set(stage, "u", u_opt[min([stage+1,N-1]), :, k])
    
            acados_ocp_solver.set(N, "x", x_opt[N, :, k])
        
        
        # update the control 
        simU[k, :] = acados_ocp_solver.solve_for_x0(simX[i, :], 
                                                    fail_on_nonzero_status=False,
                                                    print_stats_on_failure=False)

        # store CPU time required for solving the problem
        cpt[k] = acados_ocp_solver.get_stats('time_tot')

        
        # store optimal solution
        for stage in range(N):
            x_opt[stage, :, k+1] = acados_ocp_solver.get(stage, "x")
            u_opt[stage, :, k+1] = acados_ocp_solver.get(stage, "u")
            
        x_opt[N, :, k+1] = acados_ocp_solver.get(N, "x")
        
        
        # update discrete-time iteration counter
        k += 1
        
    # simulate system
    simX[i + 1, :] = acados_integrator.simulate(simX[i,:], simU[k-1, :])


ValueError: Invalid qp_solver value. Possible values are:

PARTIAL_CONDENSING_HPIPM,
FULL_CONDENSING_QPOASES,
FULL_CONDENSING_HPIPM,
PARTIAL_CONDENSING_QPDUNES,
PARTIAL_CONDENSING_OSQP,
FULL_CONDENSING_DAQP.

You have: PARTIAL_CONDENSING_QPOASES.



## PLOTS

In [ ]:
time        = np.linspace(0, ts_sim * N_steps, N_steps + 1)
time_mpc    = np.linspace(0, Ts * (N_steps_dt-1), N_steps_dt+1)

# plot the simulation results
CM = 'jet' #'inferno'
fig = plt.figure(figsize=(15, 12))
plt.subplot(5,1,1)
plt.plot(time_mpc, y_ref_nolookahead[:, 0], label='vx_ref')
plt.plot(time, simX[:, 0], label='vx')
plt.title('Longitudinal Velocity')
plt.xlabel('Time (s)')
plt.ylabel('Longitudinal Velocity (m/s)')
plt.legend()

plt.subplot(5,1,2)
plt.plot(time_mpc, y_ref_nolookahead[:, 1], label='vy_ref')
plt.plot(time, simX[:, 1], label='vy')
plt.title('Lateral Velocity')
plt.xlabel('Time (s)')
plt.ylabel('Lateral Velocity (m/s)')
plt.legend()

plt.subplot(5,1,3)
plt.plot(time_mpc, np.rad2deg(y_ref_nolookahead[:, 2]), label='r_ref')
plt.plot(time, np.rad2deg(simX[:, 2]), label='r')
plt.title('Yaw rate')
plt.xlabel('Time (s)')
plt.ylabel('Yaw rate (rad/s)')
plt.legend()

plt.subplot(5,1,4)
plt.plot(time_mpc, y_ref_nolookahead[:,3], label='Fx_ref')
plt.plot(time_mpc[:-1], simU[:, 0], label='Fx')
plt.title('Longitudinal Force')
plt.xlabel('Time (s)')
plt.ylabel('Longitudinal force (N)')
plt.ylim(-80, 80)
plt.legend()

plt.subplot(5,1,5)
plt.plot(time_mpc, np.rad2deg(y_ref_nolookahead[:,4]), label='delta_ref')
plt.plot(time_mpc[:-1], np.rad2deg(simU[:, 1]), label='delta')
plt.title('Steering angle (at the ground)')
plt.xlabel('Time (s)')
plt.ylabel('Steering angle (deg)')
plt.ylim(-20, 20)
plt.legend()


plt.suptitle('MPC simulation results', fontsize=16)
plt.tight_layout()
plt.show()